In [ ]:
# Update and import packages
!pip install -Uqr requirements.txt
!/bin/bash ./local_mode_setup.sh

# Basic packages
import json
import pandas as pd
pd.set_option("display.max_colwidth", None)

# AWS packages
import sagemaker
from sagemaker.sklearn import SKLearn
from sagemaker.predictor import Predictor
import boto3

In [ ]:
# Train SM estimator
train_data = 's3://amplifyobserverinsights-aoinsightslandingbucket29-5vcr471d4nm5/data/issues/'
sklearn_estimator = SKLearn(
    'train.py',
    role=sagemaker.get_execution_role(),
    instance_type='local',
    framework_version='0.23-1',
    hyperparameters = {'n-best': 10},
    dependencies=['.']
)
sklearn_estimator.fit({'train': train_data})

In [ ]:
# Deploy SM predictor
# predictor = sklearn_estimator.deploy(instance_type='ml.m5.4xlarge',
#                                      initial_instance_count=1,
# #                                      update_endpoint=True,
#                                     endpoint_name='issue-similarity-endpoint')
predictor = Predictor('issue-similarity-endpoint')
model = sklearn_estimator.create_model()
session = model.sagemaker_session
role = sagemaker.get_execution_role()
model_name = model.name
container_def = model.prepare_container_def(instance_type='ml.m5.4xlarge')
session.create_model(model_name, role, container_def)


endpoint_config_name = session.create_endpoint_config(name=model_name,
                                                      model_name=model_name,
                                                      initial_instance_count=1,
                                                      instance_type='ml.m5.4xlarge')

# Update desired endpoint with new Endpoint Config
client = boto3.client('sagemaker')
client.update_endpoint(EndpointName='issue-similarity-endpoint',
                       EndpointConfigName=endpoint_config_name)

# predictor.update_endpoint(
#     instance_type='ml.m5.4xlarge',
#     model_name = sklearn_estimator.create_model().name,
#     initial_instance_count=1
# )

In [ ]:
data = ['DataStore model subscription fails']
data = json.dumps({'data': data})
response = predictor.predict(data)
print(response)